In [1]:
import tensorflow as tf
import numpy as np
import samples as ls
import model_types

step_counter = 0
load_dnn = False
load_samples = False
model_num = 101
model_type = 11

model_id = '%s_%s' % (model_type, model_num)
labels,sample_ids,filename = model_types.get_labels(model_type)

if model_num == 0:
    input_dim = 6
    hidden_layers = [50,10]
elif model_num == 1:
    input_dim = 6
    hidden_layers = [50]
elif model_num == 100:
    input_dim = 6
    hidden_layers = [50]
elif model_num == 101:
    input_dim = 6
    hidden_layers = [20,20,20,20]
else:
    raise ValueError('Unknown model')

hidden_layers.insert(0,50*input_dim)
hidden_layers.append(1)
   
classes = len(labels)

#percentage used for test data
test_rate = 0.1

learning_rate = 0.01

In [2]:
import re
import math

#create dict to convert label to network output
as_angle = {}
as_degrees = {}
as_radians = {}
for label in labels:
    vals = re.match(r'angle_(\d*)(?:_(l|r))?', label).groups()
    val = float(vals[0])/180
    if vals[1] == 'r':
        val = -val
    as_angle[label] = val
    as_degrees[label] = val*360
    as_radians[label] = val*math.pi*2
print(as_angle)

{'angle_0': 0.0, 'angle_120_r': -0.6666666666666666, 'angle_60_r': -0.3333333333333333, 'angle_90_l': 0.5, 'angle_90_r': -0.5, 'angle_120_l': 0.6666666666666666, 'angle_180': 1.0, 'angle_150_r': -0.8333333333333334, 'angle_30_r': -0.16666666666666666, 'angle_60_l': 0.3333333333333333, 'angle_30_l': 0.16666666666666666, 'angle_150_l': 0.8333333333333334}


In [3]:
if load_samples:
    samples = ls.load(filename)
else:
    samples = ls.Samples(labels)

    for label in labels:
        samples.load_samples(label = label, sample_ids = sample_ids)
        
    samples.convert_to_input(size = None, accel='xyz', gyro='xyz', compass='')
    
    ls.save(samples, filename)
    
samples.unroll(steps=50) #fixed for angle samples at 50 values
samples.split_test(test_rate)
samples.flatten()

In [4]:
tf.logging.set_verbosity(tf.logging.ERROR)
tf.reset_default_graph()

xtest,ytest,_ = samples.get_test(padding=False)
xtest = np.array(xtest)
ytest = np.array(ytest)


x = tf.placeholder(tf.float32, [None, 50*input_dim], name='x')
y = tf.placeholder(tf.float32, [None], name='y')

Ws = []
for i in range(len(hidden_layers)-1):
    W = tf.Variable(tf.random_normal([hidden_layers[i], hidden_layers[i+1]]))
    b = tf.Variable(tf.zeros(hidden_layers[i+1]))
    Ws.append((W,b))


preactivations = []
activations = []
act = x
for W,b in Ws:
    preact = tf.matmul(act, W) + b
    preactivations.append(preact)
    act = tf.sigmoid(preact)
    activations.append(act)
dnn_angle = act

#simple loss function
loss = tf.reduce_mean(tf.square(tf.subtract(y, dnn_angle)))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
optimizer = tf.train.AdamOptimizer().minimize(loss)


In [5]:
sess = tf.Session()
saver = tf.train.Saver()

#summaries
def var_summary(var, name):
    mean = tf.reduce_mean(var)
    tf.summary.scalar(name + '_mean', mean)
    stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar(name + '_stddev', stddev)
    tf.summary.scalar(name + '_max', tf.reduce_max(var))
    tf.summary.scalar(name + '_min', tf.reduce_min(var))
    tf.summary.histogram(name + '_histogram', var)
i = 0
for W,b in Ws: 
    var_summary(W, 'weights_%s' % i)
    var_summary(b, 'bias_%s' % i)
    i += 1
for i in range(len(activations)):
    tf.summary.histogram('preactivations_%s' % i, preactivations[i])
    tf.summary.histogram('activations_%s' % i, activations[i])
tf.summary.scalar('loss', loss)

merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('tmp/angle/summary-%s' % model_id, sess.graph)

#init or load from checkpoint
init = tf.global_variables_initializer()
sess.run(init)

if load_dnn:
    saver.restore(sess, 'tmp/angle/model/angle-%s-%s' % (model_id,step_counter))


In [6]:
#convert label class to angle floats
def convert_to_angle(y_data):
    new_y = []
    for y in y_data:
        new_y.append(as_angle[labels[y]])
    return new_y

def test_all():
    xtest,ytest,_ = samples.get_all(use_labels_data=False)
    ytest = convert_to_angle(ytest)
    val = sess.run(loss, feed_dict={x: xtest, y: ytest})
    print("Loss: %s" % val)
    
def test():
    xtest,ytest,_ = samples.get_test(use_labels_data=False)
    ytest = convert_to_angle(ytest)
    val = sess.run(loss, feed_dict={x: xtest, y: ytest})
    print("Loss: %s" % val)

def train(steps, batch_size):
    global step_counter
    progress = 5
    for i in range(steps):
        xtrain,ytrain,_ = samples.get_batch(batch_size, use_labels_data=False)
        ytrain = convert_to_angle(ytrain)
        summary,_ = sess.run([merged, optimizer], feed_dict={x: xtrain, y: ytrain})
        train_writer.add_summary(summary, i+step_counter)
        if i>=progress/100.0*steps-1:
            print("Progress: %s%%" % progress)
            progress += 5
            test()
            saver.save(sess, 'tmp/angle/model/angle-%s' % (model_id), global_step=step_counter)
    step_counter += steps
    saver.save(sess, 'tmp/angle/model/angle-%s' % (model_id), global_step=step_counter)
    
def predict(x_input):
    return sess.run(dnn_angle, feed_dict={x:x_input})

def per_label_check():
    xtest,ytest,_ = samples.get_test(use_labels_data=False)
    ytest = convert_to_angle(ytest)
    pred = predict(xtest)
    print('Should - Is - Diff (abs)')
    for i in range(len(ytest)):
        print('%s - %s - %s' % (ytest[i], pred[i][0], abs(ytest[i]-pred[i][0])))

In [10]:
if True:
    train(10000,40)

Progress: 5%
Loss: 0.331021
Progress: 10%
Loss: 0.331021
Progress: 15%
Loss: 0.331021
Progress: 20%
Loss: 0.331021
Progress: 25%
Loss: 0.331021
Progress: 30%
Loss: 0.331021
Progress: 35%
Loss: 0.331021
Progress: 40%
Loss: 0.331021
Progress: 45%
Loss: 0.331021
Progress: 50%
Loss: 0.331021
Progress: 55%
Loss: 0.331021
Progress: 60%
Loss: 0.331021
Progress: 65%
Loss: 0.331021
Progress: 70%
Loss: 0.331021
Progress: 75%
Loss: 0.331021
Progress: 80%
Loss: 0.331021
Progress: 85%
Loss: 0.331021
Progress: 90%
Loss: 0.331021
Progress: 95%
Loss: 0.331021
Progress: 100%
Loss: 0.331021


In [8]:
if True:
    test_all()

Loss: 0.331019


In [9]:
if True:
    per_label_check()

Should - Is - Diff (abs)
0.0 - 0.0836212 - 0.0836211889982
0.0 - 0.0835618 - 0.0835617631674
0.0 - 0.0838274 - 0.0838273540139
0.0 - 0.0836359 - 0.0836359411478
0.0 - 0.0836239 - 0.083623893559
0.0 - 0.0836574 - 0.0836573764682
0.0 - 0.0836101 - 0.083610124886
0.0 - 0.0834851 - 0.0834850743413
0.0 - 0.0811718 - 0.0811718478799
0.0 - 0.0838336 - 0.0838335826993
0.0 - 0.0836637 - 0.0836637318134
0.0 - 0.0836321 - 0.0836321413517
-0.6666666666666666 - 0.0832308 - 0.749897467593
-0.6666666666666666 - 0.0837188 - 0.75038545082
-0.6666666666666666 - 0.0827837 - 0.749450410406
-0.6666666666666666 - 0.0818311 - 0.748497779171
-0.6666666666666666 - 0.0833863 - 0.750052976112
-0.6666666666666666 - 0.083779 - 0.750445658962
-0.6666666666666666 - 0.0830101 - 0.749676751594
-0.6666666666666666 - 0.0831596 - 0.749826217691
-0.6666666666666666 - 0.0830606 - 0.749727251629
-0.6666666666666666 - 0.0827033 - 0.749369921784
-0.6666666666666666 - 0.080551 - 0.747217627863
-0.6666666666666666 - 0.0839859 -